In [1]:
from pathlib import Path
import json
import pandas as pd
from datetime import datetime
from typing import List
import streamlit as st
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [2]:
DATA_PATH = Path('data') / '53526472'
MESSAGE_PATH = DATA_PATH / 'message.json'
CONVO_PATH = DATA_PATH / 'conversation.json'

with MESSAGE_PATH.open(encoding='utf-8') as f:
    msg_data = json.load(f)
    df_data_msg = pd.DataFrame(msg_data)

with CONVO_PATH.open(encoding='utf-8') as f:
    convo_data:dict = json.load(f)
    # df_convo= pd.DataFrame(convo_data)
df_data_msg.head(2)

,attachments,avatar_url,created_at,favorited_by,group_id,id,name,sender_id,sender_type,source_guid,system,text,user_id,platform,pinned_at,pinned_by,event,deleted_at,deletion_actor
0,[],https://i.groupme.com/1024x1024.jpeg.f5cfae0ce...,1668819447,[],53526472,166881944707556885,Joe Stassi,22089390,user,234392C6-8301-4FD5-AEB4-C8677CA0A802,False,thanks my man!!,22089390,gm,None,,NaN,NaN,NaN
1,"[{'loci': [[26, 11]], 'type': 'mentions', 'use...",https://i.groupme.com/960x960.jpeg.018a82eaf3b...,1668819169,"[10529832, 19901671, 22089390]",53526472,166881916925796885,J Money,12609853,user,1AF1E93C-E144-4FED-ACD6-86A33526E37B,False,Happy bday to a fine gent @Joe Stassi,12609853,gm,None,,NaN,NaN,NaN


In [3]:
members:dict = convo_data['members']
df_members = pd.DataFrame(members)
df_members.head(3)
id_map:pd.Series = df_members[['user_id','name']].set_index('user_id')['name']
df_id_map = df_members[['user_id','name']]

In [4]:
# How many muted?
# df_members[df_members.muted == True]

In [5]:
# df_data_msg[df_data_msg.system]

In [42]:
df.created_at.dt.month.apply(lambda x: calendar.month_abbr[x])

0       Nov
1       Nov
2       Nov
3       Nov
4       Nov
       ... 
4457    Sep
4458    Sep
4459    Sep
4462    Sep
4463    Sep
Name: created_at, Length: 4341, dtype: object

In [43]:
import calendar
def has_image(data:List[dict]) -> bool:
    contains_image = False   
    for dicky in data:
       if dicky['type'] == 'image':
           contains_image = True
    return contains_image
# Attachmeent ype?

# print(len(df_data_msg))
COLS_TO_KEEP = ['attachments', 'created_at', 'favorited_by', 'name',  'sender_type', 'system','text', 'user_id', 'event']
df = df_data_msg[COLS_TO_KEEP].copy()
df = df[df.system == False].copy() # Don't care about system messages for this analysis.
df['created_at'] = df.created_at.apply(datetime.utcfromtimestamp)
df['like_count'] = df.favorited_by.apply(len)
df['has_attachement'] = df.attachments.apply( lambda x: len(x) > 0)
df['attachement_num'] = df.attachments.apply( lambda x: len(x))
df['has_image'] = df.attachments.apply(has_image)
df['char_count'] = df.text.apply(lambda x: len(x) if not x is None else False )
df['date_month'] = df.created_at.dt.year.astype(str) + '-'+ df.created_at.dt.month.apply(lambda x: calendar.month_abbr[x])
# Add date cols:

df = df.merge(df_members[['user_id','nickname','name']],on='user_id',validate='m:1',how='left')

COLS_TO_DROP = ['system']
df = df.drop(columns=COLS_TO_DROP)

# Get image id
# image count
# word count
# Who is mentioned the most?
# Who mentions who the most

print(len(df))
df.head(2)

4341


,attachments,created_at,favorited_by,name_x,sender_type,text,user_id,event,like_count,has_attachement,attachement_num,has_image,char_count,date_month,nickname,name_y
0,[],2022-11-19 00:57:27,[],Joe Stassi,user,thanks my man!!,22089390,NaN,0,False,0,False,15,2022-Nov,Joe Stassi,Joe Stassi
1,"[{'loci': [[26, 11]], 'type': 'mentions', 'use...",2022-11-19 00:52:49,"[10529832, 19901671, 22089390]",J Money,user,Happy bday to a fine gent @Joe Stassi,12609853,NaN,3,True,1,False,38,2022-Nov,J Money,John Marston


In [7]:
d = df.groupby('name_y')['char_count'].mean().sort_values(ascending=False).to_frame().iloc[:10].reset_index()
d
fig = px.bar(d,x='name_y',y='char_count')

In [8]:
df[(~df.has_image) & (df.has_attachement)].iloc[2]['attachments']

[{'loci': [[0, 12]], 'type': 'mentions', 'user_ids': ['32644422']}]

In [9]:
df.groupby('name_y')['created_at'].count().sort_values(ascending=False)

name_y
Michael Hubbell       910
Sam Chastain          479
John Marston          455
Joe Cretini           343
Mary Dicharry         247
Taylor Stirling       245
Matthew Ruiz          210
Brennan Major         196
Jaxson Pinard         114
Jordi Osuna           114
Matt Morales          109
Sam Wetzel            100
Joe Stassi             97
Adam Henderson         96
Matthew Hubbell        86
Nic Brooksher          67
Pierce Hill            55
Diego Matus            50
Paul Cobler            39
Stallion Green         37
Ethan Barbin           33
William Delaune        26
Matt Comeaux           21
Graham Vogt            21
Matt Wallace           18
Christian Nunley       18
Connor Hurd            16
Easton hollis          14
James Finn             12
Andrew Searles         11
Reid Upthegrove        11
Dalton Major           10
Gregory Roy             9
Matthew Crapanzano      9
Nick Baldwin            7
Michael Ruiz            7
Terell Wilkins          7
Clayton McGee           6
Zach 

In [10]:
df[df.has_attachement == True]
df[df.attachement_num > 1].iloc[0]['attachments']

[{'type': 'image',
  'url': 'https://i.groupme.com/992x744.jpeg.6596b885bcaa4a0f8a5e9282d21f6495'},
 {'type': 'image',
  'url': 'https://i.groupme.com/1000x1332.jpeg.5e4b23bb3b75432caf554e0e33aa17c4'}]

In [11]:
df.iloc[1]['attachments']

[{'loci': [[26, 11]], 'type': 'mentions', 'user_ids': ['22089390']}]

In [28]:
df.date_month.astype(str)

0       2022-11
1       2022-11
2       2022-11
3       2022-11
4       2022-11
         ...   
4336    2019-09
4337    2019-09
4338    2019-09
4339    2019-09
4340    2019-09
Name: date_month, Length: 4341, dtype: object

In [35]:
import calendar

In [44]:
d = df.groupby('date_month').size().reset_index().rename(columns={0:'messages'})
d
px.line(d,x='date_month',y='messages')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'date_month=%{x}<br>messages=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array(['2019-Dec', '2019-Nov', '2019-Oct', '2019-Sep', '2020-Apr', '2020-Aug',
                          '2020-Dec', '2020-Feb', '2020-Jan', '2020-Jul', '2020-Jun', '2020-Mar',
                          '2020-May', '2020-Nov', '2020-Oct', '2020-Sep', '2021-Apr', '2021-Aug',
                          '2021-Dec', '2021-Feb', '2021-Jan', '2021-Jul', '2021-Jun', '2021-Mar',
                          '2021-May', '2021-Nov', '2021-Oct', '2021-Sep', '2022-Apr', '2022-Aug',
                          '2022-Feb', '2022-Jan', '2022-Jul', '2022-Jun', '2022-Mar', '2022-May',
                          '2022-Nov', '2022-Oct', '2022-Sep'], dtype=object),
              'xaxis': 'x',
              'y': array([ 75, 119,  42,  53, 222,  40,  53, 108,  85,  69,  59, 242,  41,  41,
                          133,  64,  80,  74,  93, 112, 133,  75, 131,  68,  74, 125, 158,  88,
                          294, 141, 150, 114, 189, 187, 187, 167,  26,  94, 135], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'date_month'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'messages'}}}
})

In [26]:
d = df.groupby('date_month').size()
d.index = d.index.to_timestamp()
d = d.reset_index().rename(columns={0:'messages'})
d
px.line(d,x='date_month',y='messages')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'date_month=%{x}<br>messages=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([datetime.datetime(2019, 9, 1, 0, 0),
                          datetime.datetime(2019, 10, 1, 0, 0),
                          datetime.datetime(2019, 11, 1, 0, 0),
                          datetime.datetime(2019, 12, 1, 0, 0),
                          datetime.datetime(2020, 1, 1, 0, 0),
                          datetime.datetime(2020, 2, 1, 0, 0),
                          datetime.datetime(2020, 3, 1, 0, 0),
                          datetime.datetime(2020, 4, 1, 0, 0),
                          datetime.datetime(2020, 5, 1, 0, 0),
                          datetime.datetime(2020, 6, 1, 0, 0),
                          datetime.datetime(2020, 7, 1, 0, 0),
                          datetime.datetime(2020, 8, 1, 0, 0),
                          datetime.datetime(2020, 9, 1, 0, 0),
                          datetime.datetime(2020, 10, 1, 0, 0),
                          datetime.datetime(2020, 11, 1, 0, 0),
                          datetime.datetime(2020, 12, 1, 0, 0),
                          datetime.datetime(2021, 1, 1, 0, 0),
                          datetime.datetime(2021, 2, 1, 0, 0),
                          datetime.datetime(2021, 3, 1, 0, 0),
                          datetime.datetime(2021, 4, 1, 0, 0),
                          datetime.datetime(2021, 5, 1, 0, 0),
                          datetime.datetime(2021, 6, 1, 0, 0),
                          datetime.datetime(2021, 7, 1, 0, 0),
                          datetime.datetime(2021, 8, 1, 0, 0),
                          datetime.datetime(2021, 9, 1, 0, 0),
                          datetime.datetime(2021, 10, 1, 0, 0),
                          datetime.datetime(2021, 11, 1, 0, 0),
                          datetime.datetime(2021, 12, 1, 0, 0),
                          datetime.datetime(2022, 1, 1, 0, 0),
                          datetime.datetime(2022, 2, 1, 0, 0),
                          datetime.datetime(2022, 3, 1, 0, 0),
                          datetime.datetime(2022, 4, 1, 0, 0),
                          datetime.datetime(2022, 5, 1, 0, 0),
                          datetime.datetime(2022, 6, 1, 0, 0),
                          datetime.datetime(2022, 7, 1, 0, 0),
                          datetime.datetime(2022, 8, 1, 0, 0),
                          datetime.datetime(2022, 9, 1, 0, 0),
                          datetime.datetime(2022, 10, 1, 0, 0),
                          datetime.datetime(2022, 11, 1, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([ 53,  42, 119,  75,  85, 108, 242, 222,  41,  59,  69,  40,  64, 133,
                           41,  53, 133, 112,  68,  80,  74, 131,  75,  74,  88, 158, 125,  93,
                          114, 150, 187, 294, 167, 187, 189, 141, 135,  94,  26], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'date_month'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'messages'}}}
})